# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [29]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [30]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [31]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [32]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
#with open('event_datafile_new.csv', 'r', encoding = 'utf8') as file:
 #   for line in file:
  #      print(line)


6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [33]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [34]:
# creating a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION=
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [35]:
# Setting the keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [36]:
## This is the create table statement for the 1st query. for this query sessionId and ItemInSession acts as the 
## composite primary key because just the sessionId as the primary key donot uniquely identifies the row and since the data 
## is looking for sessionid=338 and itemInSession=4, the combination of these 2 as composite primary key solves our purpose.  
query ="CREATE TABLE IF NOT EXISTS song_details_session(sessionId int,ItemInSession int, artist text,\
        song text,length float,PRIMARY KEY(sessionId,ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [37]:
# the below script opens the newly created csv file in part 1 of this notebook, reads the file and inserts the data 
#in the required columns of the table song_details_session.
file = 'event_datafile_new.csv'

with open(file,'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details_session(sessionId,ItemInSession,artist,song,length)"
        query = query + " VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [38]:
## Running the Select statement to verify that the data has been inserted and that we are getting the desired result from it.
query="SELECT artist,song,length FROM song_details_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [39]:
## This is the create table statement for the 2nd query.for this query the userid and the sessionid acts as the partition keys
## and since the query mentions, sorted by ItemInSession,ItemInSession will act as the clustering column. The 2 partition keys
## and the clustering column act as the primary key as a whole.

query ="CREATE TABLE IF NOT EXISTS song_playlist_session(userId int,sessionId int,itemInSession int,artist text\
        ,song text,firstName text,lastName text,PRIMARY KEY((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# the below script opens the newly created csv file in part 1 of this notebook, reads the file and inserts the data 
#in the required columns of the table song_playlist_session.
file = 'event_datafile_new.csv'

with open(file,'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session(userId,sessionId,itemInSession,artist,song,firstName,lastName)"
        query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

## Running the Select statement to verify that the data has been inserted and that we are getting the desired result from it
query="SELECT artist,song,firstName,lastName FROM song_playlist_session WHERE userId=10 and sessionId=182"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song,row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [40]:
## this is the create table statement for the 3rd query.for this query the data is looking for the specific song but only 
## the song is not sufficient to uniquely identify the row as there can be only 1 or many users listening to the same song.
## song in combination with userid as composite primary key uniquely identifies the row/s in the data 
query ="CREATE TABLE IF NOT EXISTS user_song(song text,userId int,firstName text,lastName text,PRIMARY KEY(song,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# the below script opens the newly created csv file in part 1 of this notebook, reads the file and inserts the data 
#in the required columns of the table user_song.
file = 'event_datafile_new.csv'

with open(file,'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song(song,userId,firstName,lastName)"
        query = query + " VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))
        
## Running the Select statement to verify that the data has been inserted and that we are getting the desired result from it
query="SELECT firstName,lastName FROM user_song WHERE song='All Hands Against His Own'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname,row.lastname)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [41]:
# Dropping the tables created above for each of the queries
query="DROP TABLE song_details_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query="DROP TABLE song_playlist_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query="DROP TABLE user_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()